In [1]:
import pandas as pd, numpy as np

In [2]:
train = pd.read_csv('data/train.csv', index_col='time')
test = pd.read_csv('data/test.csv', index_col='time')

In [3]:
X_train, y_train, X_valid, y_valid = train[[col for col in train.columns if col!='severity']], train['severity'],\
                                     test[[col for col in train.columns if col!='severity']], test['severity']

In [4]:
import spacy, string
from time import time
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

nlp = spacy.load('en_core_web_sm')

In [5]:
# Clean text before feeding it to spaCy
punctuations = string.punctuation

# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [6]:
train_cleaned = cleanup_text(X_train['message_encoding'], logging=True)

Processed 1000 out of 10364 documents.
Processed 2000 out of 10364 documents.
Processed 3000 out of 10364 documents.
Processed 4000 out of 10364 documents.
Processed 5000 out of 10364 documents.
Processed 6000 out of 10364 documents.
Processed 7000 out of 10364 documents.
Processed 8000 out of 10364 documents.
Processed 9000 out of 10364 documents.
Processed 10000 out of 10364 documents.


In [7]:
# Parse documents and print some info
print('Parsing documents...')

start = time()

train_vec = []
for doc in nlp.pipe(train_cleaned, batch_size=500):
    if doc.has_vector:
        train_vec.append(doc.vector)
    # If doc doesn't have a vector, then fill it with zeros.
    else:
        train_vec.append(np.zeros((128,), dtype="float32"))
        
# train_vec = [doc.vector for doc in nlp.pipe(train_cleaned, batch_size=500)]
train_vec = np.array(train_vec)

end = time()
print('Total time passed parsing documents: {} seconds'.format(end - start))
print('Total number of documents parsed: {}'.format(len(train_vec)))
print('Size of vector embeddings: ', train_vec.shape[1])
print('Shape of vectors embeddings matrix: ', train_vec.shape)

Parsing documents...
Total time passed parsing documents: 74.19694375991821 seconds
Total number of documents parsed: 10364
Size of vector embeddings:  128
Shape of vectors embeddings matrix:  (10364, 128)


In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, LSTM, Embedding, Bidirectional, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.optimizers import SGD

def build_model(architecture='mlp'):
    model = Sequential()
    if architecture == 'mlp':
        # Densely Connected Neural Network (Multi-Layer Perceptron)
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal', input_dim=128))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
        model.add(Dropout(0.2))
        model.add(Dense(5, activation='softmax'))
    elif architecture == 'cnn':
        # 1-D Convolutional Neural Network
        inputs = Input(shape=(128,1))

        x = Conv1D(64, 3, strides=1, padding='same', activation='relu')(inputs)

        #Cuts the size of the output in half, maxing over every 2 inputs
        x = MaxPooling1D(pool_size=2)(x)
        x = Conv1D(128, 3, strides=1, padding='same', activation='relu')(x)
        x = GlobalMaxPooling1D()(x) 
        outputs = Dense(5, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs, name='CNN')
    elif architecture == 'lstm':
        # LSTM network
        inputs = Input(shape=(128,1))

        x = Bidirectional(LSTM(64, return_sequences=True),
                          merge_mode='concat')(inputs)
        x = Dropout(0.2)(x)
        x = Flatten()(x)
        outputs = Dense(5, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs, name='LSTM')
    else:
        print('Error: Model type not found.')
    return model

/home/denis/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
from keras.utils import to_categorical

y_train_ohe = to_categorical(y_train)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_vec, y_train_ohe, test_size=0.2, random_state=21)

In [11]:
from keras import backend as K

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [12]:
# Define keras model
# Using MLP in kernel for speed
model = build_model()
# model = build_model('cnn')
#model = build_model('lstm')

# If the model is a CNN then expand the dimensions of the training data
if model.name == "CNN" or model.name == "LSTM":
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    print('Text train shape: ', X_test.shape)
    print('Text test shape: ', X_test.shape)
    
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[precision, recall, f1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               66048     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
__________

In [13]:
# Define number of epochs
epochs = 50

# Fit the model to the training data
estimator = model.fit(X_train, y_train,
                      validation_split=0.2,
                      epochs=epochs, batch_size=128, verbose=1)

Train on 6632 samples, validate on 1659 samples
Epoch 1/50
6632/6632 [==============================] - 1s 134us/step - loss: 1.3578 - precision: 0.5480 - recall: 0.3438 - f1: 0.4088 - val_loss: 1.2338 - val_precision: 0.6336 - val_recall: 0.0723 - val_f1: 0.1295
Epoch 2/50
6632/6632 [==============================] - 0s 28us/step - loss: 1.2642 - precision: 0.5630 - recall: 0.3378 - f1: 0.4022 - val_loss: 1.2420 - val_precision: 0.6823 - val_recall: 0.0494 - val_f1: 0.0917
Epoch 3/50
6632/6632 [==============================] - 0s 26us/step - loss: 1.2575 - precision: 0.5704 - recall: 0.3494 - f1: 0.4210 - val_loss: 1.1978 - val_precision: 0.5936 - val_recall: 0.4014 - val_f1: 0.4786
Epoch 4/50
6632/6632 [==============================] - 0s 26us/step - loss: 1.2423 - precision: 0.5790 - recall: 0.3593 - f1: 0.4315 - val_loss: 1.2408 - val_precision: 0.7558 - val_recall: 0.0362 - val_f1: 0.0686
Epoch 5/50
6632/6632 [==============================] - 0s 23us/step - loss: 1.2276 - preci

Epoch 39/50
6632/6632 [==============================] - 0s 29us/step - loss: 0.7851 - precision: 0.7515 - recall: 0.6125 - f1: 0.6744 - val_loss: 1.3228 - val_precision: 0.5778 - val_recall: 0.4237 - val_f1: 0.4887
Epoch 40/50
6632/6632 [==============================] - 0s 27us/step - loss: 0.7897 - precision: 0.7535 - recall: 0.6031 - f1: 0.6694 - val_loss: 1.3519 - val_precision: 0.5584 - val_recall: 0.4081 - val_f1: 0.4713
Epoch 41/50
6632/6632 [==============================] - 0s 23us/step - loss: 0.7763 - precision: 0.7517 - recall: 0.6191 - f1: 0.6782 - val_loss: 1.2986 - val_precision: 0.5963 - val_recall: 0.4551 - val_f1: 0.5158
Epoch 42/50
6632/6632 [==============================] - 0s 24us/step - loss: 0.7499 - precision: 0.7666 - recall: 0.6295 - f1: 0.6910 - val_loss: 1.3902 - val_precision: 0.5692 - val_recall: 0.4189 - val_f1: 0.4824
Epoch 43/50
6632/6632 [==============================] - 0s 24us/step - loss: 0.7239 - precision: 0.7692 - recall: 0.6350 - f1: 0.6952 -